In [1]:
from langchain_docling import DoclingLoader
from langchain_ollama import ChatOllama,OllamaEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEndpoint
from pinecone import Pinecone, ServerlessSpec
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
import torch

In [2]:
excel_file = 'new_salary.xlsx'
loader = DoclingLoader(excel_file)
docs = loader.load()
docs[:5]

Token indices sequence length is longer than the specified maximum sequence length for this model (566 > 512). Running this sequence through the model will result in indexing errors


[Document(metadata={'source': 'new_salary.xlsx', 'dl_meta': {'schema_name': 'docling_core.transforms.chunker.DocMeta', 'version': '1.0.0', 'doc_items': [{'self_ref': '#/tables/0', 'parent': {'$ref': '#/groups/0'}, 'children': [], 'content_layer': 'body', 'label': 'table', 'prov': []}], 'origin': {'mimetype': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', 'binary_hash': 14294445640614508023, 'filename': 'new_salary.xlsx'}}}, page_content='Last Name, 1 = First Name. Last Name, 2 = Status. Last Name, 3 = Salary. Williams, 1 = Mary. Williams, 2 = Full Time. Williams, 3 = 35000. Brown, 1 = Robert. Brown, 2 = Full Time. Brown, 3 = 32000. Wilson, 1 = Elizabeth. Wilson, 2 = Part Time. Wilson, 3 = 12000. Moore, 1 = Jennifer. Moore, 2 = Full Time. Moore, 3 = 41000. Brown, 1 = Charles. Brown, 2 = Full Time. Brown, 3 = 39000. Price, 1 = Lisa. Price, 2 = Part Time. Price, 3 = 14000. Wood, 1 = Daniel. Wood, 2 = Part Time. Wood, 3 = 13750. Coleman, 1 = Donald. Coleman, 2 = Full 

In [3]:
#Using Recursve Text Splitting
 
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap = 100)
split_docs = text_splitter.split_documents(docs)

In [4]:
from huggingface_hub import login
login()

In [5]:
%%cmd
pip install einops

Microsoft Windows [Version 10.0.26100.3194]
(c) Microsoft Corporation. All rights reserved.

(langchain-template) c:\Users\Ashton Internship\Desktop\Internship_Python\LLM For Excel Files\Using UnstructuredExcelLoader\Excel_Files\Docling>pip install einops

(langchain-template) c:\Users\Ashton Internship\Desktop\Internship_Python\LLM For Excel Files\Using UnstructuredExcelLoader\Excel_Files\Docling>

In [ ]:
#Create embeddings for the excel file 
# embeddings = hf_embeddings.embed_documents(split_docs)

In [5]:
load_dotenv()

True

In [ ]:
#Pinecone Database Connection
import os
PINECONE_KEY = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key = PINECONE_KEY)
print("Pinecone initialized successfully")

pcsk_exEPR_PuLKFtv2FDcw3Waf9ywQTH9d1GWtjwvXhamMZmhP7YNLSL7tdUn64XTtEXzUSPd
Pinecone initialized successfully


In [10]:
import time
index_name = "salary-db"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)
# pc.create_index(name = index_name,dimension= ) 


In [ ]:
#Qwen2-7B seems to be the best performing embedding model according to the MTEB(Massive Text Embedding Benchmark), it ranks no 1 in embedding of English and Chinese texts at the moment.

model_name = "Linq-AI-Research/Linq-Embed-Mistral"
model_kwargs = {'device': 'cuda',"trust_remote_code":True}
encode_kwargs = {'normalize_embeddings': False}
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    
)


In [ ]:
#Storing in Pinecone Vector Store
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index_name, embedding=hf_embeddings)